# Cassava Leaf Disease Classification

## Импорты

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from collections import defaultdict
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Dropout, MaxPooling2D, Conv2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications import DenseNet121, EfficientNetB5, Xception
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json


# Загрузка данных

In [ ]:
train_df = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train_path = "../input/cassava-leaf-disease-classification/train_images"
test_path = "../input/cassava-leaf-disease-classification/test_images"
train_df.head()

In [ ]:
train_df.info()

Пример фотографии

In [ ]:
img = plt.imread(train_path+"/"+train_df["image_id"][0])
print(img.shape)
plt.imshow(img)

## Первичный анализ данных

In [ ]:
file = open("../input/cassava-leaf-disease-classification/label_num_to_disease_map.json")
mapping = json.load(file)
mapping

In [ ]:
train_df.groupby('label').count().plot(kind='bar', title='Target class distribution', figsize=(20,10), grid=1)

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(20, 10))
for i, img in enumerate(train_df.groupby('label').first().reset_index().values):
    ax[i].imshow(plt.imread(train_path + f"/{img[1]}"))
    ax[i].set_title(img[0])
    ax[i].axis('off')
fig.suptitle('Image Samples', fontsize=18); 

## Подготовка данных

In [ ]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 64
INPUT_SHAPE = (IMAGE_SIZE[0], IMAGE_SIZE[0], 3)
CLASSES = 5

image_datagen = ImageDataGenerator(
    rescale=1/255.0,
    validation_split = 0.3 
)

train_df.label = train_df.label.astype('str')

In [ ]:
train_generator = image_datagen.flow_from_dataframe(
    train_df,
    directory=train_path,
    x_col="image_id",
    y_col="label",
    target_size=IMAGE_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    subset="training",
    shuffle=True,
    seed=42,
    class_mode="sparse"
)

In [ ]:
test_generator = image_datagen.flow_from_dataframe(
    train_df,
    directory=train_path,
    x_col="image_id",
    y_col="label",
    target_size=IMAGE_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    subset="validation",
    shuffle=True,
    seed=42,
    class_mode="sparse"
)

In [ ]:
example = next(train_generator)
print(example[0].shape)
plt.imshow(example[0][0,:,:,:])
plt.show()

## Создать свою модель CNN (с или без Dropout и Batch Normalization)


In [ ]:
def myCNNmodel():
    model = Sequential()

    model.add(Conv2D(16, (5,5), input_shape=INPUT_SHAPE, activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2)))
    
    model.add(Conv2D(32, (5,5), activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(64, (5,5), activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2)))
    
    model.add(Conv2D(128, (3,3), activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2)))

    model.add(Conv2D(256, (3,3), activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2)))


    model.add(Flatten())

    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.3))

    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.3))

    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.3))

    model.add(Dense(CLASSES, activation="softmax"))
    
    return model

In [ ]:
example = myCNNmodel().summary()

## Обучить с использованием разных оптимизаторов (SGD Momentum, RMSProp, Adam) с и без learning rate scheduler


Checkpoints

In [ ]:
model_checkpoint = ModelCheckpoint(
    filepath="./leafs.h5", 
    monitor='val_loss', 
    save_best_only=True, 
    verbose=1,
    mode='min')

early_stopping = EarlyStopping(
    monitor='val_loss', 
    min_delta=0,
    patience=10, 
    verbose=1, 
    restore_best_weights=True)

SGD

In [ ]:
SGD_model = myCNNmodel()
SGD_model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.SGD(), metrics=["accuracy"])

In [ ]:
%%time
SGD_history = SGD_model.fit(train_generator,
                validation_data = test_generator,
                steps_per_epoch = train_generator.n // BATCH_SIZE,
                validation_steps = test_generator.n // BATCH_SIZE,
                epochs=10,
                callbacks=[model_checkpoint, early_stopping])

RMSPropp

In [ ]:
RMS_model = myCNNmodel()
RMS_model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.RMSprop(), metrics=["accuracy"])

In [ ]:
%%time
RMS_history = RMS_model.fit(train_generator,
                validation_data = test_generator,
                steps_per_epoch = train_generator.n // BATCH_SIZE,
                validation_steps = test_generator.n // BATCH_SIZE,
                epochs=10,
                callbacks=[model_checkpoint, early_stopping])

Adam

In [ ]:
Adam_model = myCNNmodel()
Adam_model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

In [ ]:
%%time
Adam_history = Adam_model.fit(train_generator,
                validation_data = test_generator,
                steps_per_epoch = train_generator.n // BATCH_SIZE,
                validation_steps = test_generator.n // BATCH_SIZE,
                epochs=10,
                callbacks=[model_checkpoint, early_stopping])

Adam лучше всех

Подберем оптимальный learning rate с помощью меньшего набора данных

In [ ]:
train_generator_tiny = image_datagen.flow_from_dataframe(
    train_df[:1000],
    directory=train_path,
    x_col="image_id",
    y_col="label",
    target_size=IMAGE_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    subset="training",
    shuffle=True,
    seed=42,
    class_mode="sparse"
)
test_generator_tiny = image_datagen.flow_from_dataframe(
    train_df[:1000],
    directory=train_path,
    x_col="image_id",
    y_col="label",
    target_size=IMAGE_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    subset="validation",
    shuffle=True,
    seed=42,
    class_mode="sparse"
)

In [ ]:
lr_rate = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))

In [ ]:
LRS_model = myCNNmodel()
LRS_model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-8), metrics=["accuracy"])

In [ ]:
%%time
LRS_history = LRS_model.fit(train_generator_tiny,
                validation_data = test_generator_tiny,
                steps_per_epoch = train_generator_tiny.n // BATCH_SIZE,
                validation_steps = test_generator_tiny.n // BATCH_SIZE,
                epochs=100,
                callbacks=[lr_rate])

In [ ]:
plt.semilogx(LRS_history.history["lr"], LRS_history.history["loss"])

Лучший learning_rate - 1e-4 

In [ ]:
LR_model = myCNNmodel()
LR_model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"])

In [ ]:
%%time
LR_history = LR_model.fit(train_generator,
                validation_data = test_generator,
                steps_per_epoch = train_generator.n // BATCH_SIZE,
                validation_steps = test_generator.n // BATCH_SIZE,
                epochs=10)

## С и без использованием аугментации изображений

In [ ]:
image_datagen = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=5,
    zoom_range=0.1,
    shear_range=0.05,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = image_datagen.flow_from_dataframe(
    train_df,
    directory=train_path,
    x_col="image_id",
    y_col="label",
    target_size=IMAGE_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    subset="training",
    shuffle=True,
    seed=42,
    class_mode="sparse"
)

test_generator = image_datagen.flow_from_dataframe(
    train_df,
    directory=train_path,
    x_col="image_id",
    y_col="label",
    target_size=IMAGE_SIZE,
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    subset="validation",
    shuffle=True,
    seed=42,
    class_mode="sparse"
)

In [ ]:
example = next(train_generator)
print(example[0].shape)
plt.imshow(example[0][0,:,:,:])
plt.show()

In [ ]:
example = next(train_generator)
print(example[0].shape)
plt.imshow(example[0][0,:,:,:])
plt.show()

In [ ]:
example = next(train_generator)
print(example[0].shape)
plt.imshow(example[0][0,:,:,:])
plt.show()

In [ ]:
AUG_model = myCNNmodel()
AUG_model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"])

In [ ]:
%%time
AUG_history = AUG_model.fit(train_generator,
                validation_data = test_generator,
                steps_per_epoch = train_generator.n // BATCH_SIZE,
                validation_steps = test_generator.n // BATCH_SIZE,
                epochs=10)

## Отобразить активации внутренних слоев сети или сделать визуализацию фильтров 

Визуализация фильтров

In [ ]:
for layer in Adam_model.layers:
    if 'conv' not in layer.name:
        continue
    filters, biases = layer.get_weights()
    print(layer.name, filters.shape)

In [ ]:
from matplotlib import pyplot
# load the model
model = Adam_model

for layer in Adam_model.layers:
    if 'conv' not in layer.name:
        continue
    filters, biases = layer.get_weights()
    f_min, f_max = filters.min(), filters.max()
    filters = (filters - f_min) / (f_max - f_min)
    # plot first few filters
    n_filters, ix = 6, 1
    for i in range(n_filters):
        # get the filter
        f = filters[:, :, :, i]
        # plot each channel separately
        for j in range(3):
            # specify subplot and turn of axis
            ax = pyplot.subplot(n_filters, 3, ix)
            ax.set_xticks([])
            ax.set_yticks([])
            # plot filter channel in grayscale
            pyplot.imshow(f[:, :, j])
            ix += 1
    # show the figure
    pyplot.show()

## Transfer Learning: Использовать предобученную модель на ImageNet (ResNet, VGG, GoogLeNet и т д) с заменой слоя классификации на слой с нужным количеством классов и обучить модель на данном датасете


In [ ]:
base_model = DenseNet121(include_top=True, input_shape=(224,224,3))

newLayer = Dense(CLASSES, activation='softmax')
outLayer = newLayer(base_model.layers[-2].output)

Dense_model = Model(inputs=base_model.input, outputs=outLayer)

for layer in base_model.layers[:-1]:
    layer.trainable = False
    
Dense_model.summary()

In [ ]:
Dense_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
%%time
Dense_history = Dense_model.fit(
    train_generator,
    validation_data = test_generator,
    epochs = 10
)

## Отобразить результаты всех экспериментов в одной таблице(пример дальше) и графики обучения

In [ ]:
ALL_accuracy = [SGD_history.history['accuracy'][-1],
                RMS_history.history['accuracy'][-1],
                Adam_history.history['accuracy'][-1],
                LR_history.history['accuracy'][-1],
                AUG_history.history['accuracy'][-1], 
                Dense_history.history['accuracy'][-1]]

ALL_val_accuracy = [SGD_history.history['val_accuracy'][-1],
                RMS_history.history['val_accuracy'][-1],
                Adam_history.history['val_accuracy'][-1],
                LR_history.history['val_accuracy'][-1],
                AUG_history.history['val_accuracy'][-1],
                Dense_history.history['val_accuracy'][-1]]

ALL_loss = [SGD_history.history['loss'][-1],
                RMS_history.history['loss'][-1],
                Adam_history.history['loss'][-1],
                LR_history.history['loss'][-1],
                AUG_history.history['loss'][-1],
                Dense_history.history['loss'][-1]]

ALL_val_loss = [SGD_history.history['val_loss'][-1],
                RMS_history.history['val_loss'][-1],
                Adam_history.history['val_loss'][-1],
                LR_history.history['val_loss'][-1],
                AUG_history.history['val_loss'][-1],
                Dense_history.history['val_loss'][-1]]

In [ ]:
experiments = {"experiment": ["Custom model + SGD", "Custom model + RMS", "Custom model + Adam", "Custom model + Adam + LearningRate", "Custom model + Adam + LR + Image augment", "DenseNet + Adam"],
              "train_accuracy": ALL_accuracy,
              "test_accuracy": ALL_val_accuracy}

In [ ]:
table = pd.DataFrame(experiments)
table

In [ ]:
def plot_results(model_hist):
    plt.figure(figsize=(20,10))

    x = model_hist.epoch

    plt.title('accuracy')
    plt.subplot(221)
    plt.plot(x, model_hist.history['accuracy'])

    plt.title('val_accuracy')
    plt.subplot(222)
    plt.plot(x, model_hist.history['val_accuracy'])

    plt.title('loss')
    plt.subplot(223)
    plt.plot(x, model_hist.history['loss'])

    plt.title('val_loss')
    plt.subplot(224)
    plt.plot(x, model_hist.history['val_loss'])

    plt.plot()

In [ ]:
plot_results(Dense_history)